In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
btvote = pd.read_pickle('../data/btvote_encoded.pkl')
btvote.head()

,party,vote_19001,vote_19002,vote_19003,vote_19004,vote_19005,vote_19006,vote_19007,vote_19008,vote_19009,...,vote_19235,vote_19236,vote_19237,vote_19238,vote_19239,vote_19240,vote_19241,vote_19242,vote_19243,vote_19244
0,CDU/CSU,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN
2,Linke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
3,CDU/CSU,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN
4,Linke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,NaN


# Split data and encode target variable

In [3]:
# Split dataframe in 'data' and 'target'
btvote_data = btvote.drop('party', axis=1)
btvote_target = btvote['party']

# Encode the target variable
label_encoder = preprocessing.LabelEncoder()
btvote_target = label_encoder.fit_transform(btvote_target)

# Pipeline and GridSearch setup

In the pipeline we include the KNNImputer with n_neighbors=1 and don't do any balancing as a result of the experiments in the previous notebooks.\
As an estimator, we first consider:
- k-NearestNeighbor
- NearestCentroid
- Decision Tree
- Naive Bayes

In [4]:
from sklearn.pipeline import Pipeline
# imputer
from sklearn.impute import KNNImputer
# classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# create Pipeline
pipeline = Pipeline([('imputer', KNNImputer(n_neighbors=1)), ('estimator', None)])

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# specify the cross validation
stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# define the scoring function
# Note: We use 'macro' as average as we want to evaluate the performance of each class equally, regardless of the class size
f1 = make_scorer(f1_score, average='macro')

# k-NN vs. Nearest Centroid

In [6]:
# set parameter grid
parameters = [
    {
        'estimator': [KNeighborsClassifier()],
        'estimator__n_neighbors': range(2, 10),
        'estimator__weights': ['uniform', 'distance'],
        'estimator__p': [1, 2]  # For Minkowski distance (1 for Manhattan, 2 for Euclidean)'
    }, {
        'estimator': [NearestCentroid()],
        'estimator__metric': ['manhattan', 'euclidean']
    }
]

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['classifier'] = results['param_estimator'].astype(str)
results['n_neighbors'] = results['param_estimator__n_neighbors'].astype(str)
results['weights'] = results['param_estimator__weights'].astype(str)
results.loc[results['classifier']=='KNeighborsClassifier()', 'metric'] = results['param_estimator__p'].replace({1:'manhattan', 2:'euclidean'})
results.loc[results['classifier']=='NearestCentroid()', 'metric'] = results['param_estimator__metric']
results['ranking'] = results['rank_test_score'].astype(int)
pivoted_results = results.pivot(index=['classifier','n_neighbors','weights'], columns='metric', values=['mean_test_score', 'ranking'])
pivoted_results['ranking'] = pivoted_results['ranking'].astype(int)
display(pivoted_results)

mean_test_score            \
metric                                            euclidean manhattan   
classifier             n_neighbors weights                              
KNeighborsClassifier() 2           distance        0.847908  0.848283   
                                   uniform         0.829633  0.831856   
                       3           distance        0.861415  0.863383   
                                   uniform         0.861415  0.862419   
                       4           distance        0.857324  0.859950   
                                   uniform         0.839420  0.839669   
                       5           distance        0.873762  0.870913   
                                   uniform         0.871810  0.869071   
                       6           distance        0.873536  0.873646   
                                   uniform         0.868562  0.867051   
                       7           distance        0.866883  0.864281   
                                   uniform         0.865278  0.862752   
                       8           distance        0.864672  0.863919   
                                   uniform         0.855271  0.852768   
                       9           distance        0.866204  0.863868   
                                   uniform         0.863164  0.861129   
NearestCentroid()      nan         nan             0.863948  0.849471   

                                              ranking            
metric                                      euclidean manhattan  
classifier             n_neighbors weights                       
KNeighborsClassifier() 2           distance        30        29  
                                   uniform         34        33  
                       3           distance        21        17  
                                   uniform         21        20  
                       4           distance        25        24  
                                   uniform         32        31  
                       5           distance         1         5  
                                   uniform          4         6  
                       6           distance         3         2  
                                   uniform          7         8  
                       7           distance         9        13  
                                   uniform         11        19  
                       8           distance        12        15  
                                   uniform         26        27  
                       9           distance        10        16  
                                   uniform         18        23  
NearestCentroid()      nan         nan             14        28

The F1-score is the highest for the k-NN with n_neighbors between 5 and 6. We will further analyse this special case. The NearestCentroid results lie in somewhere in between all k-NN results, so we will neglect the NearestCentroid classifier from now on. 

In [7]:
# set parameter grid
parameters = {
    'estimator': [KNeighborsClassifier()],
    'estimator__n_neighbors': range(5, 7),
    'estimator__weights': ['uniform', 'distance'],
    'estimator__p': [1, 2]  # For Minkowski distance (1 for Manhattan, 2 for Euclidean)
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['n_neighbors'] = results['param_estimator__n_neighbors'].astype(str)
results['weights'] = results['param_estimator__weights'].astype(str)
results['metric'] = results['param_estimator__p'].replace({1:'manhattan', 2:'euclidean'})
pivoted_results = results.pivot(index=['n_neighbors','weights'], columns='metric', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc[('Average',''),:] = pivoted_results.mean()
display(pivoted_results)

metric                euclidean  manhattan   Average
n_neighbors weights                                 
5           distance   0.873762   0.870913  0.872338
            uniform    0.871810   0.869071  0.870441
6           distance   0.873536   0.873646  0.873591
            uniform    0.868562   0.867051  0.867807
Average                0.871918   0.870170  0.871044

The results for the k-NN Classifier are better when using distance-based weights. Also, the euclidean distance performs at least as good as the manhatten distance in all cases. For that specific case we will do a nested cross validation for the two options of n_neighbors to receive more validated results:

In [8]:
from sklearn.model_selection import cross_val_score

# We use stratified 10 fold cross validation for the inner and the outer fold.
print('Nested Cross Validation Mean scores:')

# 5-NN
pipeline.set_params(estimator=KNeighborsClassifier(n_neighbors=5))
print('- 5-NN: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# 6-NN
pipeline.set_params(estimator=KNeighborsClassifier(n_neighbors=6))
print('- 6-NN: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

Nested Cross Validation Mean scores:
- 5-NN: 0.8718104364349765
- 6-NN: 0.8685620498968648


The 5-NN claassifier seems to perform slightly better than the one with n_neighbors=6. So, from now on we will use the default parameter n_neighbors=5.

# DecisionTree analysis

In [9]:
# set parameter grid
parameters = {
    'estimator': [DecisionTreeClassifier()],
    'estimator__criterion': ['gini', 'entropy'],
    'estimator__max_depth': [None, 5, 10, 15],
    'estimator__min_samples_split': [2, 5, 10],
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['criterion'] = results['param_estimator__criterion'].astype(str)
results['max_depth'] = results['param_estimator__max_depth'].astype(str)
results['min_samples_split'] = results['param_estimator__min_samples_split']
pivoted_results = results.pivot(index=['criterion','max_depth'], columns='min_samples_split', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc[('Average',''),:] = pivoted_results.mean()
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_2524\4012610856.py:22: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index=['criterion','max_depth'], columns='min_samples_split', values='mean_test_score')


min_samples_split           2         5        10   Average
criterion max_depth                                        
entropy   10         0.839935  0.840000  0.839729  0.839888
          15         0.848722  0.853737  0.846568  0.849676
          5          0.833253  0.834352  0.822050  0.829885
          None       0.853859  0.842905  0.850494  0.849086
gini      10         0.839241  0.833848  0.838786  0.837292
          15         0.859708  0.858579  0.857437  0.858575
          5          0.840745  0.832310  0.836550  0.836535
          None       0.853977  0.851867  0.850347  0.852063
Average              0.846180  0.843450  0.842745  0.844125

Observations:
- max_depth: choosing a max_depth of 5 and 10 works rather badly. For 15 and None, the results are similar.
- min_samples_split: On average, there are no real differences, 10 works slightly less good.
- criterion: For a larger max_depth, the top values can be achieved for the gini index.

We choose criterion='gini' (default), max_depth=15 and min_samples_split=2 (default) as the best parameter setting for a DecisionTreeClassifier.

# NearestCentroid vs. DecisionTree vs. NaiveBayes

In [11]:
# set parameter grid
parameters = {
    'estimator': [KNeighborsClassifier(),
                  DecisionTreeClassifier(max_depth=15),
                  GaussianNB()
                  ]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results[['param_estimator','mean_test_score','std_test_score','rank_test_score']])

,param_estimator,mean_test_score,std_test_score,rank_test_score
0,KNeighborsClassifier(),0.871810,0.025288,1
1,DecisionTreeClassifier(max_depth=15),0.858583,0.028441,2
2,GaussianNB(),0.855503,0.025973,3


The Grid Search cross validation shows a slight advantage for the 5-NN classifier, while the DecisionTreeClassifier and the NaiveBayes approach are close together in second and third.
We will validate the results by performing a *nested* cross validation for each of the appraoches.

In [12]:
# We use stratified 10 fold cross validation for the inner and the outer fold.
print('Nested Cross Validation Mean scores:')

# NearestCentroid
pipeline.set_params(estimator=KNeighborsClassifier())
print('- 5-NN: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# NearestCentroid
pipeline.set_params(estimator=DecisionTreeClassifier(max_depth=15))
print('- DecisionTree: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# NearestCentroid
pipeline.set_params(estimator=GaussianNB())
print('- NaiveBayes: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

Nested Cross Validation Mean scores:
- 5-NN: 0.8718104364349765
- DecisionTree: 0.8580977411092597
- NaiveBayes: 0.8555026293412993


We see our results from before validated.

# Summary

Until now, we evaluated:
- k-NearestNeighbor
- NearestCentroid
- Decision Tree
- Naive Bayes

We've seen that out of these four model families, the k-NN Classifier with n_neighbors=5 performs the best with a F1-score of approx. 0.87

# Extended model evaluation

We will now compare the 5-NN Classifier to some more complex approaches, that aren't dealt with in detail in the lecture slides or the exercises.

## Neural network: Multi-Layer Perceptron

In [13]:
from sklearn.neural_network import MLPClassifier

# set parameter grid
parameters = {
    'estimator': [MLPClassifier()],
    'estimator__hidden_layer_sizes': [(50), (50, 50), (100, 50, 25), (100, 50, 25, 10)], # Specify the architecture of hidden layers
    'estimator__alpha': [0.0001, 0.001, 0.01], # L2 regularization term
    'estimator__max_iter': [300], # Increase number of gradient steps so that all combinations convert
    'estimator__random_state': [42],  # Random seed for reproducibility
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['hidden_layer_sizes'] = results['param_estimator__hidden_layer_sizes'].astype(str)
results['alpha'] = results['param_estimator__alpha']
pivoted_results = results.pivot(index='hidden_layer_sizes', columns='alpha', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc['Average',:] = pivoted_results.mean()
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_2524\2553089787.py:24: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index='hidden_layer_sizes', columns='alpha', values='mean_test_score')


alpha,0.0001,0.001,0.01,Average
hidden_layer_sizes,,,,
"(100, 50, 25)",0.849769,0.856335,0.846362,0.850822
"(100, 50, 25, 10)",0.832572,0.844697,0.858612,0.845294
"(50, 50)",0.843736,0.842481,0.847272,0.844496
50,0.867829,0.869065,0.861735,0.866210
Average,0.848477,0.853144,0.853495,0.851705


We see that using three (or generally more) hidden layers doesn't lead to a higher F1-Score. The best results can be achieved for a single hidden layer of size 50. In that case, the alpha parameter 0.001 produces the best results with a score of approx. 0.869.

## RandomForestClassifier

At first, we analyse how the criterion and the number of trees affect the result:

In [14]:
from sklearn.ensemble import RandomForestClassifier

# set parameter grid
parameters = {
    'estimator': [RandomForestClassifier()],
    'estimator__criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    'estimator__n_estimators': [10, 50, 100, 150, 200],  # Number of trees in the forest
    'estimator__random_state': [42]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['criterion'] = results['param_estimator__criterion'].astype(str)
results['n_estimators'] = results['param_estimator__n_estimators']
pivoted_results = results.pivot(index='criterion', columns='n_estimators', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc['Average',:] = pivoted_results.mean()
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_2524\3257803325.py:23: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index='criterion', columns='n_estimators', values='mean_test_score')


n_estimators,10,50,100,150,200,Average
criterion,,,,,,
entropy,0.859083,0.863847,0.862187,0.867689,0.869622,0.864486
gini,0.865170,0.867407,0.865956,0.869484,0.863504,0.866304
Average,0.862126,0.865627,0.864072,0.868587,0.866563,0.865395


We see that neither the criterion nor the n_estimators effectively changes the result. According to Occam's Razor we will take the combination ('gini', 50) as it produces the best result and is relatively simple. Now, we will use this pair to evaluate the effect of max_depth and min_samples_split.

In [15]:
# set parameter grid
parameters = {
    'estimator': [RandomForestClassifier()],
    'estimator__criterion': ['gini'],
    'estimator__n_estimators': [50],
    'estimator__max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'estimator__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'estimator__random_state': [42]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['max_depth'] = results['param_estimator__max_depth']
results['min_samples_split'] = results['param_estimator__min_samples_split']
pivoted_results = results.pivot(index='max_depth', columns='min_samples_split', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc['Average',:] = pivoted_results.mean()
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_2524\2865267119.py:23: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index='max_depth', columns='min_samples_split', values='mean_test_score')


min_samples_split,2,5,10,Average
max_depth,,,,
NaN,0.867407,0.867659,0.866584,0.867217
10.0,0.856474,0.855458,0.853264,0.855065
20.0,0.861573,0.861907,0.858684,0.860721
30.0,0.866644,0.864605,0.867172,0.866140
Average,0.863025,0.862407,0.861426,0.862286


The results don't differ much for different min_samples_split values, so we will keep the default value 2. A larger max_depth seams to increase the F1-score, but we will further analyse this now.

In [16]:
# set parameter grid
parameters = {
    'estimator': [RandomForestClassifier()],
    'estimator__criterion': ['gini'],
    'estimator__n_estimators': [50],
    'estimator__max_depth': [10, 20, 30, 40, 50, 60, None],  # Maximum depth of the trees
    'estimator__random_state': [42]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results[['param_estimator__max_depth','mean_test_score','std_test_score','rank_test_score']])

,param_estimator__max_depth,mean_test_score,std_test_score,rank_test_score
0,10,0.856474,0.032736,7
1,20,0.861573,0.038289,6
2,30,0.866644,0.033805,4
3,40,0.865563,0.038027,5
4,50,0.867407,0.036381,1
5,60,0.867407,0.036381,1
6,None,0.867407,0.036381,1


Above a max_depth of 40 the result cannot be improved any more. So, we will take max_depth=40 as the best parameter.

## Summary

We finally perform a nested cross validation for MLP and RandomForestClassifier to compare it to the best k-NN Classifier.

In [17]:
# We use stratified 10 fold cross validation for the inner and the outer fold.
print('Nested Cross Validation Mean scores:')

# 5-NN
pipeline.set_params(estimator=KNeighborsClassifier(n_neighbors=5))
print('- 5-NN: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# MLPClassifier
pipeline.set_params(estimator=MLPClassifier(hidden_layer_sizes=(50), alpha=0.001, max_iter=300, random_state=42))
print('- MLPClassifier: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# RandomForestClassifier
pipeline.set_params(estimator=RandomForestClassifier(criterion='gini', n_estimators=50, max_depth=40, random_state=42))
print('- RandomForestClassifier: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

Nested Cross Validation Mean scores:
- 5-NN: 0.8718104364349765
- MLPClassifier: 0.8690646524220599
- RandomForestClassifier: 0.8655630892306547


The 5-NN Classifier achieves the highest F1-Score with 0.87. Close behind we have the MLPClassifier and lastly the RandomForestClassifier.